In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import importlib
import itertools
import matplotlib
matplotlib.use('Agg')
import time
from   multiprocessing import Pool
import numpy as np
import os
import pdb
import pickle
import subprocess
import sys
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import tf_slim as slim
import threading
import scipy.misc
from skimage import color
from taskonomy.taskbank.lib.models.sample_models import *
from taskonomy.taskbank.lib.data.synset import *
import scipy
import skimage
import skimage.io
import taskonomy.code.tools.init_paths
import transforms3d
import math
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from taskonomy.taskbank.tools.task_viz import *
import random
import taskonomy.taskbank.tools.utils as utils
import models.architectures as architectures
from   data.load_ops import resize_rescale_image
from   data.load_ops import rescale_image
import lib.data.load_ops as load_ops

os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [ ]:
imagenet_validation_path = '/mnt/raid/data/ni/dnn/ILSVRC2012_img_val'
meta_file_path = '/mnt/raid/ni/agnessa/RSA/'
layers_path = '/mnt/raid/ni/agnessa/RSA/layer_names'
ROOT_PATH = '/mnt/raid/ni/agnessa/RSA/Edge_detection/'


# Select Data and get Metadata
Select 10 images of each of the 1000 classes of the validation data set together with their label. 

In [ ]:
class ILSVRCSubDataset(Dataset):
    """ILSVRC 2012 subset of the original val dataset"""

    def __init__(self, json_file, root, transform=None):
        """
        Args:
            json_file (string): Path to the json file with meta.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root = root
        self.transform = transform
        
        # Parse META File
        with open(json_file, "r") as fd:
            self.meta = json.load(fd)
        print(self.meta)

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        path = os.path.join(self.root,
                            self.meta[idx]["0"]) #merge root and the filename of the sample
        sample = default_loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
            
        wnid = self.meta[idx]["1"]
            
        return sample, wnid #sample, class

In [ ]:
# data_transforms = transforms.Compose([
#     transforms.Resize(255),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225])
# ])

dataset_val = ILSVRCSubDataset(json_file=os.path.join(meta_file_path,'meta.json'),
                               root=imagenet_validation_path,
                               transform=None) #double-check that these transforms are not needed

#find a way to iterate in tensorflow
# dataloaders = torch.utils.data.DataLoader(dataset_val, #Combines a dataset and a sampler, and provides an iterable over the given dataset.
#                                           batch_size=20, #how many samples per batch to load
#                                           shuffle=False)


# Get the activations from a layer for all samples and save them
Use the subset with 10 images of 1000 classes on torchvisions pretrained models, get the activations of specific layers and calculate the Input RDM by correlating between the activations.

## Define functions to create filenames

In [1]:
def getFileName(n_samples, name):
    return name \
        + "_{}_".format(n_samples) \
        + "_{}_".format(model_name) \
        + "_{}".format(layer_name)  \
        + ".npy"        

## Get activations

In [ ]:
#load json file with the layers of interest 
# model_name = 'googlenet'
json_file_layers=os.path.join(layers_path,model_name + '_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)
layer_name = selected_layers[12].get('layer') #increase the index by one each time you restart the kernel


tf.compat.v1.disable_eager_execution()

#replace by data iterator
# img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
# store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 
# task = 'edge2d'

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def generate_cfg(task):
    
    repo_dir = '/mnt/raid/ni/agnessa/RSA/Edge_detection/taskonomy/taskbank/'
    CONFIG_DIR = os.path.join(repo_dir, 'experiments/final', task)
    
    ############## Load Configs ##############
    import taskonomy.taskbank.tools.utils as utils
    import data.load_ops as load_ops
    from   general_utils import RuntimeDeterminedEnviromentVars
    cfg = utils.load_config( CONFIG_DIR, nopause=True )
    RuntimeDeterminedEnviromentVars.register_dict( cfg )
    cfg['batch_size'] = 1
    if 'batch_size' in cfg['encoder_kwargs']:
        cfg['encoder_kwargs']['batch_size'] = 1
    cfg['model_path'] = os.path.join( repo_dir, 'temp', task, 'model.permanent-ckpt' )
    cfg['root_dir'] = repo_dir
    return cfg

def run_to_task():
    import general_utils
    from   general_utils import RuntimeDeterminedEnviromentVars

    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    
    #input info
    NR_OF_SAMPLES = 10000 #num classes*num samples per class;  len(dataset_val)   
    batch_size = 20
    
    #replace by data iterator
    img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
    task = 'edge2d'
    store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 

    img = load_raw_image_center_crop( img )
    img = Image.fromarray(np.squeeze(img))
    cfg = generate_cfg(task)

    # Since we observe that areas with pixel values closes to either 0 or 1 sometimes overflows, we clip pixels value
    cfg['input_preprocessing_fn'] = load_ops.resize_rescale_image_low_sat
    img = cfg[ 'input_preprocessing_fn' ]( img, **cfg['input_preprocessing_fn_kwargs'] )
    img = img[np.newaxis,:]
    print("Doing {task}".format(task=task))
    general_utils = importlib.reload(general_utils)
    
    from tensorflow.python.framework import ops
    ops.reset_default_graph()

    training_runners = { 'sess': tf.compat.v1.InteractiveSession(), 'coord': tf.train.Coordinator() }

    ############## Set Up Inputs ##############
    # tf.logging.set_verbosity( tf.logging.INFO )
    setup_input_fn = utils.setup_input
    inputs = setup_input_fn( cfg, is_training=False, use_filename_queue=False )
    RuntimeDeterminedEnviromentVars.load_dynamic_variables( inputs, cfg )
    RuntimeDeterminedEnviromentVars.populate_registered_variables()
    start_time = time.time()

    ############## Set Up Model ##############
    model = utils.setup_model( inputs, cfg, is_training=False )
    m = model[ 'model' ]
    model[ 'saver_op' ].restore( training_runners[ 'sess' ], cfg[ 'model_path' ] )
    
    init = tf.compat.v1.global_variables_initializer()
    layer_name = 'encoder/block2/unit_1/bottleneck_v1/conv1/Conv2D'
    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        sess.graph.get_operations()
        layer = sess.graph.get_tensor_by_name(f'{layer_name}:0')#,encoder/block2/unit_1/bottleneck_v1/conv1/Conv2D:0')
        print('got layer')
    
        representation = training_runners['sess'].run( 
            [ layer ], feed_dict={m.input_images: img} )
        print('got representation')
    

#         s_name, file_extension = os.path.splitext(args.store_name)
#         with open('{}.npy'.format(s_name), 'wb') as fp:
#             np.save(fp, np.squeeze(representation))



#         just_rescale = ['autoencoder', 'denoise', 'edge2d', 
#                     'edge3d', 'keypoint2d', 'keypoint3d',
#                     'reshade', 'rgb2sfnorm' ]

#         if task in just_rescale:
#             simple_rescale_img(predicted, args.store_name)
#             return
    

    ############## Clean Up ##############
    training_runners[ 'coord' ].request_stop()
    training_runners[ 'coord' ].join()
    print("Done: {}".format(config_name))

    ############## Reset graph and paths ##############            
    tf.reset_default_graph()
    training_runners['sess'].close()
    return

if __name__ == '__main__':
    run_to_task()



# Option 2 - check if this works

In [ ]:
def generate_cfg(task):
    
    repo_dir = '/mnt/raid/ni/agnessa/RSA/Edge_detection/taskonomy/taskbank/'
    CONFIG_DIR = os.path.join(repo_dir, 'experiments/final', task)
    
    ############## Load Configs ##############
    import taskonomy.taskbank.tools.utils as utils
    import data.load_ops as load_ops
    from   general_utils import RuntimeDeterminedEnviromentVars
    cfg = utils.load_config( CONFIG_DIR, nopause=True )
    RuntimeDeterminedEnviromentVars.register_dict( cfg )
    cfg['batch_size'] = 1
    if 'batch_size' in cfg['encoder_kwargs']:
        cfg['encoder_kwargs']['batch_size'] = 1
    cfg['model_path'] = os.path.join( repo_dir, 'temp', task, 'model.permanent-ckpt' )
    cfg['root_dir'] = repo_dir
    return cfg

In [ ]:
#load json file with the layers of interest 
# model_name = 'googlenet'
json_file_layers=os.path.join(layers_path,model_name + '_selected_layers.json')
with open(json_file_layers, "r") as fd:
    selected_layers = json.load(fd)
layer_name = selected_layers[12].get('layer') #increase the index by one each time you restart the kernel


tf.compat.v1.disable_eager_execution()

#replace by data iterator
# img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
# store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 
# task = 'edge2d'

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


import general_utils
from   general_utils import RuntimeDeterminedEnviromentVars

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#input info
NR_OF_SAMPLES = 10000 #num classes*num samples per class;  len(dataset_val)   
batch_size = 20

#replace by data iterator
img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
task = 'edge2d'
store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 

img = load_raw_image_center_crop( img )
img = Image.fromarray(np.squeeze(img))
cfg = generate_cfg(task)

# Since we observe that areas with pixel values closes to either 0 or 1 sometimes overflows, we clip pixels value
cfg['input_preprocessing_fn'] = load_ops.resize_rescale_image_low_sat
img = cfg[ 'input_preprocessing_fn' ]( img, **cfg['input_preprocessing_fn_kwargs'] )
img = img[np.newaxis,:]
print("Doing {task}".format(task=task))
general_utils = importlib.reload(general_utils)

from tensorflow.python.framework import ops
ops.reset_default_graph()

training_runners = { 'sess': tf.compat.v1.InteractiveSession(), 'coord': tf.train.Coordinator() }

############## Set Up Inputs ##############
# tf.logging.set_verbosity( tf.logging.INFO )
setup_input_fn = utils.setup_input
inputs = setup_input_fn( cfg, is_training=False, use_filename_queue=False )
RuntimeDeterminedEnviromentVars.load_dynamic_variables( inputs, cfg )
RuntimeDeterminedEnviromentVars.populate_registered_variables()
start_time = time.time()

############## Set Up Model ##############
model = utils.setup_model( inputs, cfg, is_training=False )
m = model[ 'model' ]
model[ 'saver_op' ].restore( training_runners[ 'sess' ], cfg[ 'model_path' ] )

init = tf.compat.v1.global_variables_initializer()

layer_name = 'encoder/block2/unit_1/bottleneck_v1/conv1/Conv2D'
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    sess.graph.get_operations()
    layer = sess.graph.get_tensor_by_name(f'{layer_name}:0')#,encoder/block2/unit_1/bottleneck_v1/conv1/Conv2D:0')
    print('got layer')
    
    #insert the image loop
    #put the img transforms here? look in the VGG16 script to see the pipeline
    representation = training_runners['sess'].run( 
        [ layer ], feed_dict={m.input_images: img} )
    print('got representation')


#         s_name, file_extension = os.path.splitext(args.store_name)
#         with open('{}.npy'.format(s_name), 'wb') as fp:
#             np.save(fp, np.squeeze(representation))



#         just_rescale = ['autoencoder', 'denoise', 'edge2d', 
#                     'edge3d', 'keypoint2d', 'keypoint3d',
#                     'reshade', 'rgb2sfnorm' ]

#         if task in just_rescale:
#             simple_rescale_img(predicted, args.store_name)
#             return


############## Clean Up ##############
training_runners[ 'coord' ].request_stop()
training_runners[ 'coord' ].join()
print("Done: {}".format(config_name))

############## Reset graph and paths ##############            
tf.reset_default_graph()
training_runners['sess'].close()


if __name__ == '__main__':
    run_to_task()

